# Lab 5

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib as mp
import matplotlib.pyplot as plt 
import scipy as sci
import skimage as ski
import PIL as pil

from PIL import Image
from scipy.ndimage import gaussian_filter
from skimage import data
from skimage import data, img_as_float
from skimage.morphology import reconstruction
from skimage.segmentation import chan_vese
from scipy import stats
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

In [2]:
D0 = pd.read_csv('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 5 - Culturing Hybirdoma Cells and Immodetection/Data/ELISA.csv',
                header=0)

SD = plt.imread('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 5 - Culturing Hybirdoma Cells and Immodetection/Data/SDS.tiff',)

WB = plt.imread('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 5 - Culturing Hybirdoma Cells and Immodetection/Data/WB.tiff')

In [3]:
D0.T

,0,1,2,3,4,5,6,7
Dilution,Mouse mAb alpha-HT 1,Mouse mAb alpha-HT 2,Mouse mAb alpha-HT sample 1,Mouse mAb alpha-HT sample 2,Mouse mAb alpha-OTI 1,Mouse mAb alpha-OTI 2,Rat mAb alpha-HT 1,Rat mAb alpha-HT 2
1,0.628,0.621,0.598,0.57,0.139,0.135,0.294,0.303
2,0.632,0.574,0.552,0.538,0.14,0.137,0.303,0.273
4,0.575,0.568,0.458,0.472,0.138,0.139,0.272,0.273
8,0.48,0.456,0.374,0.388,0.14,0.138,0.283,0.285
16,0.367,0.373,0.273,0.271,0.145,0.139,0.269,0.251
32,0.264,0.259,0.207,0.206,0.14,0.138,0.216,0.192
64,0.214,0.209,0.175,0.177,0.141,0.141,0.198,0.182
128,0.174,0.171,0.157,0.155,0.139,0.138,0.155,0.153
256,0.154,0.15,0.145,0.144,0.137,0.136,0.143,0.144


In [4]:
MH = pd.DataFrame(D0.iloc[0:2,:].mean(axis=0))
MH.columns = ['MH']

MS = pd.DataFrame(D0.iloc[2:4,:].mean(axis=0))
MS.columns = ['MS']

MO = pd.DataFrame(D0.iloc[4:6,:].mean(axis=0))
MO.columns = ['MO']

RH = pd.DataFrame(D0.iloc[6:8,:].mean(axis=0))
RH.columns = ['RH']

In [5]:
DT = pd.concat([MH, MS, MO, RH], sort=False, axis=1)
DT.columns = [r'Mouse mAb $\alpha$-HT (+ Control)', 
              r'Mouse mAb $\alpha$-HT (Sample)', 
              r'Mouse mAb $\alpha$-OTI (- Control)', 
              r'Rat mAb $\alpha$-HT (- Control)']
DT

,Mouse mAb $\alpha$-HT (+ Control),Mouse mAb $\alpha$-HT (Sample),Mouse mAb $\alpha$-OTI (- Control),Rat mAb $\alpha$-HT (- Control)
1,0.6245,0.5840,0.1370,0.2985
2,0.6030,0.5450,0.1385,0.2880
4,0.5715,0.4650,0.1385,0.2725
8,0.4680,0.3810,0.1390,0.2840
16,0.3700,0.2720,0.1420,0.2600
32,0.2615,0.2065,0.1390,0.2040
64,0.2115,0.1760,0.1410,0.1900
128,0.1725,0.1560,0.1385,0.1540
256,0.1520,0.1445,0.1365,0.1435
512,0.1435,0.1410,0.1355,0.1395


In [6]:
slope, intercept, r_value, p_value, std_err = stats.linregress([3,4], DT.iloc[3:5,1])
y = DT[r'Mouse mAb $\alpha$-HT (Sample)'].max()/2
x = (y - intercept)/slope

## SDS-Page

In [94]:
def plot_fig(A, res=1200, conv=25.4, gridstep=5, figsize=(5, 6), cmap=None,
             vmin=None, vmax=None):    
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.imshow(A, aspect='equal', cmap=cmap,
              vmin=vmin, vmax=vmax,
              extent=(0, np.shape(A)[1]*conv/res, 
                      np.shape(A)[0]*conv/res, 0))

    ax.yaxis.tick_right()
    ax.yaxis.set_label_position("right")
    ax.set_xticks(np.arange(0, np.shape(A)[1]*conv/res, step=gridstep))
    ax.set_yticks(np.arange(0, np.shape(A)[0]*conv/res, step=gridstep))
    # plt.yticks(fontsize=24)
    plt.grid(linewidth=1, linestyle='--')
    shape = np.shape(A)
    return shape

In [95]:
plot_fig(SD)

(2973, 2453, 3)

<Figure size 360x432 with 1 Axes>

In [77]:
# Dilated 
SD_gray = ski.color.rgb2gray(np.array(SD))

seed_SDg = np.copy(SD_gray)
seed_SDg[1:-300, 180:-500] = SD_gray.min()
mask_SDg = SD_gray

dilated_SDg = reconstruction(seed_SDg, mask_SDg, method='dilation')

In [114]:
plot_fig(np.exp(dilated_SDg), cmap='gray', vmax=2.5, vmin=1.25)

(2973, 2453)

<Figure size 360x432 with 1 Axes>

## Western Blot

In [116]:
plot_fig(WB[:-200, :], figsize=(5, 10))

(4297, 2436, 3)

<Figure size 360x720 with 1 Axes>